<a href="https://colab.research.google.com/github/duyvm/leetcode-problems/blob/main/%5BHAR%5D2818_Apply_Operations_to_Maximize_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2818. Apply Operations to Maximize Score

https://leetcode.com/problems/apply-operations-to-maximize-score/description/

You are given an array `nums` of `n` positive integers and an integer `k`.

Initially, you start with a score of `1`. You have to maximize your score by applying the following operation at most `k` times:

- Choose any **non-empty** subarray `nums[l, ..., r]` that you haven't chosen previously.

- Choose an element `x` of `nums[l, ..., r]` with the highest **prime score**. If multiple such elements exist, choose the one with the smallest index.

- Multiply your score by `x`.

Here, `nums[l, ..., r]` denotes the subarray of `nums` starting at index `l` and ending at the index `r`, both ends being **inclusive**.

The prime score of an integer `x` is equal to the number of distinct prime factors of `x`. For example, the prime score of `300` is `3` since `300 = 2 * 2 * 3 * 5 * 5`.

Return the **maximum** possible score after applying at most `k` operations.

Since the answer may be large, return it modulo `10`<sup>`9`</sup>` + 7`.

**Constraints:**
- `1 <= nums.length == n <= 10`<sup>`5`</sup>
- `1 <= nums[i] <= 10`<sup>`5`</sup>
- `1 <= k <= min(n * (n + 1) / 2, 10`<sup>`9`</sup>`)`

**Example 1:**

- Input: `nums = [8,3,9,3,8], k = 2`

- Output: `81`

- Explanation:

        To get a score of 81, we can apply the following operations:
        - Choose subarray nums[2, ..., 2]. nums[2] is the only element in this subarray. Hence, we multiply the score by nums[2]. The score becomes 1 * 9 = 9.
        - Choose subarray nums[2, ..., 3]. Both nums[2] and nums[3] have a prime score of 1, but nums[2] has the smaller index. Hence, we multiply the score by nums[2]. The score becomes 9 * 9 = 81.
        It can be proven that 81 is the highest score one can obtain.

**Example 2:**

- Input: `nums = [19,12,14,6,10,18], k = 3`

- Output: `4788`

- Explanation:

        To get a score of 4788, we can apply the following operations:
        - Choose subarray nums[0, ..., 0]. nums[0] is the only element in this subarray. Hence, we multiply the score by nums[0]. The score becomes 1 * 19 = 19.
        - Choose subarray nums[5, ..., 5]. nums[5] is the only element in this subarray. Hence, we multiply the score by nums[5]. The score becomes 19 * 18 = 342.
        - Choose subarray nums[2, ..., 3]. Both nums[2] and nums[3] have a prime score of 2, but nums[2] has the smaller index. Hence, we multipy the score by nums[2]. The score becomes 342 * 14 = 4788.
        It can be proven that 4788 is the highest score one can obtain.

In [51]:
test_cases = [
    {
        "input": {
            "nums": [8,3,9,3,8],
            "k": 2,
        },
        "output": 81
    },
    {
        "input": {
            "nums": [19,12,14,6,10,18],
            "k": 3,
        },
        "output": 4788
    },
    {
        "input": {
            "nums": [3289,2832,14858,22011],
            "k": 6,
        },
        "output": 256720975
    },
    {
        "input": {
            "nums": [92820,1,89870,22094,1,80617,55614,18696,30916,98455,46185,1,79019,7665,36572,1,49394,57640,51870,3990,62790,52883,1,41845,59023,1,69320,85470,74343,7712,79111,57144],
            "k": 82,
        },
        "output": 857810838
    },
]

In [2]:
def run_test_cases(solution, function_name):
    for i in range(len(test_cases)):
        run_test_case(solution, function_name, i)

def run_test_case(solution, function_name, i):
    test_case = test_cases[i]
    result = getattr(solution, function_name)(**test_case["input"])
    if result != test_case["output"]:
        print(f'Failed test case {i} with input: {test_case["input"]} and expected result: {test_case["output"]} vs actual result: {result}')

# Approach 1 - Time Limit Exceeded

### Observations

- How to find prime score of an integer `x`:

 - Try to divide `x` by number in range of `[2:sqrt(x)+1]` inclusive.

   - If there is number that `x mod number = 0`, remove all occurrences of `number` in `x` first by `x //= number`

   - If `x` still larger or equal `2` after loop over above array, then left over `x` is a prime number

 - Let `s` is the pre-compute prime score array of all elements in `nums`

- The idea is: for an element `x`, we compute all **subarray** that `x` is the highest prime score. Let `n` is the number of such subarrays. The score for all such subarrays is `x * n`.

- With this, we can start to compute score of subarrays that have the maximum value `x`<sub>`max`</sub> as highest prime score.
  
  - If `k - n > 0`, we compute `score *= x ** n`, then continue with second largest element and `k = k - n` and so on

  - If `k - n = 0`, we stop with `score *= x ** n`

  - If `k -n < 0`, we stop with `score *= x ** k`

- To efficiently compute `n` of `x` at index `i`:

  - Check the left of `i`: If `s[i-1] < s[i]`, we continue with `i-2` and so on, until `s[i-left] >= s[i]`, we stop. Let `left` is the total number of elements that are in the left of `nums[i]` and have prime score smaller than `nums[i]`.

  - Check the right of `i`: If `s[i+1] <= s[i]`, we continue with `i+2` and so on, until `s[i+right] > s[i]`, we stop. Let `right` is the total number of elements that are in the right of `nums[i]` and have prime score smaller or equal to `nums[i]`.

  - The total number of subarrays that have `nums[i]` is the higest prime score is computed as follow:

        left = number of subarrays starting from nums[i] to the left: nums[i], nums[i-1:i+1], nums[i-2:i+1] ...
        
        right = number of subarrays starting from nums[i] to the right: nums[i], nums[i:i+2], nums[i:i+3] ...
        
        Because both left and right have duplicated nums[i] subarray, number of subarrays starting from i to left and right is: left + right - 1
        
        We also need to consider the combination of subarrays in the left (exclude nums[i]) with subarrays in the right (exclude nums[i]): (left - 1) * (right - 1)
        
        With above observations, we have:

        n = (left + right - 1) + (left - 1) * (right - 1)
          =  left + right - 1 + left*right - left - right + 1
          = left * right


### Analysis

- Time complexity:

### Implementation

In [87]:
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:
    def maximumScore(self, nums: List[int], k: int) -> int:
        desc_sorted_nums_with_index = sorted(enumerate(nums), key=lambda x: x[1], reverse=True)
        s = [self.getPrimeScore(num) for num in nums]
        score = 1
        for idx, num in desc_sorted_nums_with_index:
            if k == 0:
                break
            n = self.computeNumSubarrays(idx, nums, s)
            if k - n >= 0:
                score *= num ** n
                k -= n
            elif k - n < 0:
                score *= num ** k
                k = 0

        return score % (10**9 + 7)

    def computeNumSubarrays(self, current_idx: int, nums: List[int], s: List[int]) -> int:
        left = 1
        i = 1
        while True:
            if current_idx - i >= 0 and s[current_idx - i] < s[current_idx]:
                left += 1
                i += 1
            else:
                break

        right = 1
        i = 1
        while True:
            if current_idx + i < len(nums) and s[current_idx + i] <= s[current_idx]:
                right += 1
                i += 1
            else:
                break

        return left * right



    def getPrimeScore(self, num: int) -> int:
        prime_score = 0
        for factor in range(2, int(math.sqrt(num)) + 1):
            if num % factor == 0:
                prime_score += 1

                while num % factor == 0:
                    num //= factor

        # If num is not change, it's a prime factor
        if num >= 2:
            prime_score += 1

        return prime_score

In [88]:
run_test_cases(Solution(), "maximumScore")

In [89]:
run_test_case(Solution(), "maximumScore", 1)

# Approach 2 - Optimized approach 1 - Time Limit Exceeded

### Additional optimizations

- Using modular exponentiation to compute large number

### Analysis

- Time complexity:

In [90]:
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:
    def maximumScore(self, nums: List[int], k: int) -> int:
        self.MOD = 10**9 + 7
        desc_sorted_nums_with_index = sorted(enumerate(nums), key=lambda x: x[1], reverse=True)
        s = [self.getPrimeScore(num) for num in nums]
        score = 1
        for idx, num in desc_sorted_nums_with_index:
            if k == 0:
                break
            n = self.computeNumSubarrays(idx, nums, s)
            if k - n >= 0:
                score = score * self._power(num, n) % self.MOD
                k -= n
            elif k - n < 0:
                score = score * self._power(num, k) % self.MOD
                break

        return score

    def computeNumSubarrays(self, current_idx: int, nums: List[int], s: List[int]) -> int:
        left = 1
        i = 1
        while True:
            if current_idx - i >= 0 and s[current_idx - i] < s[current_idx]:
                left += 1
                i += 1
            else:
                break

        right = 1
        i = 1
        while True:
            if current_idx + i < len(nums) and s[current_idx + i] <= s[current_idx]:
                right += 1
                i += 1
            else:
                break

        return left * right

    def getPrimeScore(self, num: int) -> int:
        prime_score = 0
        for factor in range(2, int(math.sqrt(num)) + 1):
            if num % factor == 0:
                prime_score += 1

                while num % factor == 0:
                    num //= factor

        # If num is not change, it's a prime factor
        if num >= 2:
            prime_score += 1

        return prime_score

    def _power(self, base, exponent):
        res = 1

        # Calculate the exponentiation using binary exponentiation
        while exponent > 0:
            # If the exponent is odd, multiply the result by the base
            if exponent % 2 == 1:
                res = (res * base) % self.MOD

            # Square the base and halve the exponent
            base = (base * base) % self.MOD
            exponent //= 2

        return res

In [91]:
run_test_cases(Solution(), "maximumScore")

# Approach 3 - Optimized approach 2 + Monotonic stack (Beat ~ 42%)

### Additional optimizations

- Using Monotonic stack to pre-compute the index of higher prime score elements that are in left and right of an element at index `i`

  - Let `leftHigherScore[i]` is the index of element on the left side of `nums[i]` that has higher prime score than `nums[i]`. We have `left = i - leftHigherScore[i]`

  - Let `rightHigherScore[i]` is the index of element on the right side of `nums[i]` that has higher prime score than `nums[i]`. We have `right = rightHigerScore[i] - i`

  - Compute `leftHigherScore` and `rightHigherScore` using Monotonic stack

### Analysis

- Time complexity:

In [101]:
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:
    def maximumScore(self, nums: List[int], k: int) -> int:
        self.MOD = 10**9 + 7
        desc_sorted_nums_with_index = sorted(enumerate(nums), key=lambda x: x[1], reverse=True)
        score = 1
        left, right = self.computeLeftRigthHigherScoreArray(nums)
        for idx, num in desc_sorted_nums_with_index:
            if k == 0:
                break
            n = self.computeNumSubarrays(idx, left, right)
            if k - n >= 0:
                score = score * self._power(num, n) % self.MOD
                k -= n
            elif k - n < 0:
                score = score * self._power(num, k) % self.MOD
                break

        return score

    def computeLeftRigthHigherScoreArray(self, nums):
        s = [self.getPrimeScore(num) for num in nums]
        leftHigherScore = [-1] * len(s)
        rightHigherScore = [len(s)] * len(s)
        stack = [ (0, s[0]) ]
        for i in range(1, len(s)):
            while stack and s[i] > stack[-1][1]:
                last_idx, last_score = stack.pop()
                rightHigherScore[last_idx] = i

            if stack:
                leftHigherScore[i] = stack[-1][0]
            stack.append((i, s[i]))

        return leftHigherScore, rightHigherScore

    def computeNumSubarrays(self, current_idx, leftHigherScore, rightHigherScore) -> int:
        left = current_idx - leftHigherScore[current_idx]

        right = rightHigherScore[current_idx] - current_idx

        return left * right

    def getPrimeScore(self, num: int) -> int:
        prime_score = 0
        for factor in range(2, int(math.sqrt(num)) + 1):
            if num % factor == 0:
                prime_score += 1

                while num % factor == 0:
                    num //= factor

        # If num is not change, it's a prime factor
        if num >= 2:
            prime_score += 1

        return prime_score

    def _power(self, base, exponent):
        res = 1

        # Calculate the exponentiation using binary exponentiation
        while exponent > 0:
            # If the exponent is odd, multiply the result by the base
            if exponent % 2 == 1:
                res = (res * base) % self.MOD

            # Square the base and halve the exponent
            base = (base * base) % self.MOD
            exponent //= 2

        return res

In [102]:
run_test_case(Solution(), "maximumScore", -1)